In [ ]:
import requests
from bs4 import BeautifulSoup

# from google.colab import files

url_req = (['?imei=359632102795827&s_dt=2022-06-28%2010:31:00&e_dt=2022-06-28%2011:25:00',
            '?imei=352094089278805&s_dt=2022-06-28%2010:31:00&e_dt=2022-06-28%2011:25:00'])

web_link = 'https://nav.by/info/testAccel.php'

db_files = []

### postman
import requests

url = "https://nav.by/login_get_data.php?type=LOGIN&data%5Blogin%5D=diler_el_vek&data%5Bpwd%5D=111111&data%5BscreenWidth%5D=1920&data%5BscreenHeight%5D=1080"

payload = {}
headers = {
    'Cookie': 'pc_id=60b9bbc0-bef7-4c4b-9052-003672c5d1a9; ses_id=15fb4b00-046b-4e59-bd88-175381dd03b8'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


### postman

def remove_str(value, deletechars_):
    for c in deletechars_:
        value = value.replace(c, '_')
    return value


NUM_OF_FILES = len(url_req)
for p in range(NUM_OF_FILES):

    name_of_file = remove_str(url_req[p], '\/:*?"<>|&%=-')
    db_files.append(name_of_file)

    print(name_of_file)

    file = open(name_of_file + ".csv", "w")
    file.close()

    response = requests.request("GET", web_link + url_req[p], headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'lxml')

    file = open(name_of_file + ".csv", "a+")

    quotes = soup.find_all('p')
    for quote in quotes:
        file.write(quote.text)
    file.close()

    file = open(name_of_file + ".csv", "r")
    lines = file.readlines()
    file.close()

    f_first_line = 1

    file = open(name_of_file + ".csv", "w")
    for line in lines:
        if f_first_line == 1:
            file.write(line)
            f_first_line = 0
        if ((
                line != "date;speed;78;64011;64022;64033;64044;delta64011;delta64022;delta64033;delta64044;vodo;lat;lon;dir;alt;engine_speed;distance_gps;201;202;203;204;210;211;212;213;214;215;ain1;ain2\n") and (
                line != "\n")):
            file.write(line)
    file.close()

In [ ]:
import math
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from array import array
sns.set(style="darkgrid")

In [ ]:
from scipy.optimize import curve_fit

# initial guesses for k1, k2, k3, k4, k5:
kfc_init = 0., 0., 0., 0., 0.


def func_fuel_consumption(x, k1, k2, k3, k4, k5):
    delta_es_app_, delta_es_2_app_, delta_es_app_2_, delta_es_2_app_2_ = x
    return k1 + k2 * delta_es_app_ + k3 * delta_es_2_app_ + k4 * delta_es_app_2_ + k5 * delta_es_2_app_2_


In [ ]:
NUM_OF_FILES = len(db_files)

num_points = [0] * NUM_OF_FILES

delta_ES = []
delta_ES_APP = []
delta_ES_2_APP = []
delta_ES_APP_2 = []
delta_ES_2_APP_2 = []
delta_level_dut = []
delta_level_dut_calc = []
delta_level_dut_calc2 = []
delta_level_dut_ctrl = []
eps_level = []
eps2_level = []

popt_calc = []
popt_calc2 = []
matrix_linear = []

for p in range(NUM_OF_FILES):
    df = pd.read_csv(db_files[p] + '.csv', sep=';', engine='python', decimal=',')
    df.columns = ['date', 'speed', 'rowdata', 'z', 'y', 'x_m', 'x_p', 'delta64011', 'delta64022', 'delta64033',
                  'delta64044', 'vodo', 'lat', 'lon', 'dir', 'alt', 'engine_speed', 'distance_gps', 'AnIn3',
                  'mark_ind1', 'sum_ES_APP', 'mark_ind2', 'sum_ES_2_APP', 'mark_ind3', 'sum_ES_APP_2', 'mark_ind4',
                  'sum_ES_2_APP_2', 'mark_ind5', 'AnIn3', 'AnIn2']

    print('len(' + db_files[p] + '):' + str(len(df)))

    sum_ES_sum = 0
    sum_ES_APP_sum = 0
    sum_ES_2_APP_sum = 0
    sum_ES_APP_2_sum = 0
    sum_ES_2_APP_2_sum = 0

    AnIn1_begin = 0

    sum_ES_prev = 0
    sum_ES_APP_prev = 0
    sum_ES_2_APP_prev = 0
    sum_ES_APP_2_prev = 0
    sum_ES_2_APP_2_prev = 0
    AnIn1_prev = 0

    k = 1

    for i in range(1, len(df)):

        if df.sum_ES_APP[i] != 0 and df.sum_ES_APP[i] == df.sum_ES_APP[i] and df.sum_ES_APP[i] > sum_ES_APP_prev:
            sum_ES_APP_sum += df.sum_ES_APP[i] - sum_ES_APP_prev
            sum_ES_APP_prev = df.sum_ES_APP[i]

        if df.sum_ES_2_APP[i] != 0 and df.sum_ES_2_APP[i] == df.sum_ES_2_APP[i] and df.sum_ES_2_APP[
            i] > sum_ES_2_APP_prev:
            sum_ES_2_APP_sum += df.sum_ES_2_APP[i] - sum_ES_2_APP_prev
            sum_ES_2_APP_prev = df.sum_ES_2_APP[i]

        if df.sum_ES_APP_2[i] != 0 and df.sum_ES_APP_2[i] == df.ssum_ES_APP_2[i] and df.sum_ES_APP_2[
            i] > sum_ES_APP_2_prev:
            sum_ES_APP_2_sum += df.sum_ES_APP_2[i] - sum_ES_APP_2_prev
            sum_ES_APP_2_prev = df.sum_ES_APP_2[i]

        if df.sum_ES_2_APP_2[i] != 0 and df.sum_ES_2_APP_2[i] == df.sum_ES_2_APP_2[i] and df.sum_ES_2_APP_2[
            i] > sum_ES_2_APP_2_prev:
            sum_ES_2_APP_2_sum += df.sum_ES_2_APP_2[i] - sum_ES_2_APP_2_prev
            sum_ES_2_APP_2_prev = df.sum_ES_2_APP_2[i]

        if df.AnIn1[i] != 0 and df.AnIn1[i] == df.AnIn1[i] and df.AnIn1[i] != 'x':
            AnIn1_prev = df.AnIn1[i]
            if AnIn1_begin == 0:
                AnIn1_begin = df.AnIn1[i]
            if AnIn1_begin != 0 and (AnIn1_begin - AnIn1_prev) > 5 * k:
                k += 1
                #delta_ES.append(sum_ES_sum)
                delta_ES_APP.append(sum_ES_APP_sum)
                delta_ES_2_APP.append(sum_ES_2_APP_sum)
                delta_ES_APP_2.append(sum_ES_APP_2_sum)
                delta_ES_2_APP_2.append(sum_ES_2_APP_2_sum)
                matrix_linear.append([delta_ES_APP[-1], delta_ES_2_APP[-1], delta_ES_APP_2[-1], delta_ES_2_APP_2[-1]])
                delta_level_dut.append(AnIn1_prev - AnIn1_begin)
                if p == 1:
                    delta_level_dut_ctrl.append(AnIn1_begin - AnIn1_prev)
                    delta_level_dut_calc.append(func_fuel_consumption(
                        (delta_ES_APP[-1], delta_ES_2_APP[-1], delta_ES_APP_2[-1], delta_ES_2_APP_2[-1]), popt_calc[0],
                        popt_calc[1], popt_calc[2], popt_calc[3], popt_calc[4]))
                    delta_level_dut_calc2.append(func_fuel_consumption(
                        (delta_ES_APP[-1], delta_ES_2_APP[-1], delta_ES_APP_2[-1], delta_ES_2_APP_2[-1]), popt_calc2[0],
                        popt_calc2[1], popt_calc2[2], popt_calc2[3], popt_calc2[4]))
    if p == 0:
        popt, pcov = curve_fit(func_fuel_consumption, (delta_ES_APP, delta_ES_2_APP, delta_ES_APP_2, delta_ES_2_APP_2),
                               delta_level_dut, kfc_init)
        print(popt)
        popt_calc = popt
        popt_calc2 = np.linalg.solve(np.array(matrix_linear), np.array(delta_level_dut))
    if p == 1:
        plt.figure(figsize=(14, 10))
        plt.plot(delta_level_dut_ctrl, 'b')
        plt.plot(delta_level_dut_calc, 'r')
        plt.plot(delta_level_dut_calc2, 'r--')
        plt.show()

        for jg in range(len(delta_level_dut_ctrl)):
            eps_level.append((delta_level_dut_ctrl[jg] - delta_level_dut_calc[jg]) / delta_level_dut_ctrl[jg])
            eps2_level.append((delta_level_dut_ctrl[jg] - delta_level_dut_calc2[jg]) / delta_level_dut_ctrl[jg])

        plt.figure(figsize=(14, 10))
        plt.plot(eps_level, 'b')
        plt.plot(eps2_level, 'r')
        plt.show()
